In [1]:
import numpy as np
import pandas as pd
from deltalake import DeltaTable
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

/mnt/Data Volume/Projects/Storion/Codebase/storion_aggregator_engine/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
tbl = DeltaTable('../data/articles')

In [3]:
df = tbl.to_pandas()

In [4]:
df.head()

,id,title,author,publication_date,source,url,summary,content,tags,categories,images
0,672960439902a3465058b1f0,Datadog challenger Dash0 aims to dash observab...,"Anna Heim ,Devin Coldewey ,Marina Temkin ,Maxw...",2024-11-04T00:00:00.000000,ArticleSource(id=ObjectId('6696759a347e0ad7140...,https://techcrunch.com/2024/11/04/datadog-chal...,,The end of zero-interest rates has driven comp...,"Fundraising ,Startups ,observability ,cloud co...",NULL,https://techcrunch.com/wp-content/uploads/2024...
1,67295ff69902a3465058b1ef,Karen tries to claim ownership of the place sh...,NULL,NULL,ArticleSource(id=ObjectId('6696759a347e0ad7140...,https://cheezburger.com/37613061/karen-tries-t...,,Scroll down for the next article,"entitled parents ,housing ,Random Memes ,Geek ...",NULL,https://i.chzbgr.com/full/10424000000/hAC1368F...
2,67295f349902a3465058b1ee,Elon Musk's ex Grimes takes swipe at Tesla bil...,"Eve Buckland ,Eve Buckland For Dailymail.Com ,...",2024-11-04T21:53:57.000000+0000,ArticleSource(id=ObjectId('6696759a347e0ad7140...,https://www.dailymail.co.uk/tvshowbiz/article-...,,Elon Musk's ex Grimes took a savage swipe at t...,NULL,NULL,https://i.dailymail.co.uk/1s/2024/11/03/05/916...
3,67295f059902a3465058b1ed,Kansas City Chiefs find success in bringing ba...,"Adam Teicher ,Jenna Laine ,Todd Archer ,Kather...",NULL,ArticleSource(id=ObjectId('6696759a347e0ad7140...,https://www.espn.com/nfl/story/_/id/42171653/k...,,Get ready for an electric Week 9 Monday Night ...,NULL,NULL,https://a2.espncdn.com/combiner/i?img=%2Fphoto...
4,67295dd59902a3465058b1ec,CMS’s Medical Debt Relief Will Worsen Medical ...,"Ge Bai ,Ge Baicontributoropinions Expressed Fo...",2024-11-04T00:00:00.000000,ArticleSource(id=ObjectId('6696759a347e0ad7140...,https://www.forbes.com/sites/gebai/2024/11/04/...,,Man collects money with magnet from human crow...,NULL,NULL,https://specials-images.forbesimg.com/imageser...


In [5]:
df.count()

id                  2200
title               2200
author              2200
publication_date    2200
source              2200
url                 2200
summary             2200
content             2200
tags                2200
categories          2200
images              2200
dtype: int64

In [6]:
df.columns

Index(['id', 'title', 'author', 'publication_date', 'source', 'url', 'summary',
       'content', 'tags', 'categories', 'images'],
      dtype='object')

In [7]:
df.drop(columns=['id']).duplicated().sum()

np.int64(120)

In [18]:
df.set_index('id', inplace=True)

In [9]:
df.head()

,title,author,publication_date,source,url,summary,content,tags,categories,images
id,,,,,,,,,,
672960439902a3465058b1f0,Datadog challenger Dash0 aims to dash observab...,"Anna Heim ,Devin Coldewey ,Marina Temkin ,Maxw...",2024-11-04T00:00:00.000000,ArticleSource(id=ObjectId('6696759a347e0ad7140...,https://techcrunch.com/2024/11/04/datadog-chal...,,The end of zero-interest rates has driven comp...,"Fundraising ,Startups ,observability ,cloud co...",NULL,https://techcrunch.com/wp-content/uploads/2024...
67295ff69902a3465058b1ef,Karen tries to claim ownership of the place sh...,NULL,NULL,ArticleSource(id=ObjectId('6696759a347e0ad7140...,https://cheezburger.com/37613061/karen-tries-t...,,Scroll down for the next article,"entitled parents ,housing ,Random Memes ,Geek ...",NULL,https://i.chzbgr.com/full/10424000000/hAC1368F...
67295f349902a3465058b1ee,Elon Musk's ex Grimes takes swipe at Tesla bil...,"Eve Buckland ,Eve Buckland For Dailymail.Com ,...",2024-11-04T21:53:57.000000+0000,ArticleSource(id=ObjectId('6696759a347e0ad7140...,https://www.dailymail.co.uk/tvshowbiz/article-...,,Elon Musk's ex Grimes took a savage swipe at t...,NULL,NULL,https://i.dailymail.co.uk/1s/2024/11/03/05/916...
67295f059902a3465058b1ed,Kansas City Chiefs find success in bringing ba...,"Adam Teicher ,Jenna Laine ,Todd Archer ,Kather...",NULL,ArticleSource(id=ObjectId('6696759a347e0ad7140...,https://www.espn.com/nfl/story/_/id/42171653/k...,,Get ready for an electric Week 9 Monday Night ...,NULL,NULL,https://a2.espncdn.com/combiner/i?img=%2Fphoto...
67295dd59902a3465058b1ec,CMS’s Medical Debt Relief Will Worsen Medical ...,"Ge Bai ,Ge Baicontributoropinions Expressed Fo...",2024-11-04T00:00:00.000000,ArticleSource(id=ObjectId('6696759a347e0ad7140...,https://www.forbes.com/sites/gebai/2024/11/04/...,,Man collects money with magnet from human crow...,NULL,NULL,https://specials-images.forbesimg.com/imageser...


In [5]:
df.duplicated().sum()

np.int64(120)

In [5]:
df.drop_duplicates(keep='last', inplace=True)

In [7]:
df.count()

title               2080
author              2080
publication_date    2080
source              2080
url                 2080
summary             2080
content             2080
tags                2080
categories          2080
images              2080
dtype: int64

Extract article source string properly

In [5]:
df['source'].iloc[0]

"ArticleSource(id=ObjectId('6696759a347e0ad714039daa'), name='TechCrunch', domain='techcrunch.com', rss_url='https://techcrunch.com/feed/', categories=['Top Sources', 'Technology'])"

In [6]:
type(df['source'].iloc[0])

str

In [7]:
df['source'].iloc[0].split(',')[1][7:-1]

'TechCrunch'

In [8]:
df['source'].str.split(',').str[1].str[7:-1]

id
672960439902a3465058b1f0            TechCrunch
67295ff69902a3465058b1ef             FAIL Blog
67295f349902a3465058b1ee            Daily Mail
67295f059902a3465058b1ed                  ESPN
67295dd59902a3465058b1ec                Forbes
                                   ...        
6724ba569aa896701328e024            Refinery29
6724ba559aa896701328e023                   Vox
6724ba559aa896701328e022    The New York Times
6724ba549aa896701328e021             IndieWire
6724ba539aa896701328e020         Rolling Stone
Name: source, Length: 2200, dtype: object

In [6]:
df['source'] = df['source'].str.split(',').str[1].str[7:-1]

Set datetime format for publication_date

In [7]:
df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce')

## Selecting sample cases
- First let's manually select some sample news events which can be used for the consolidation task

In [13]:
# Load 5 rows with the word 'giveaway' in the content from df
df[df['content'].str.contains('giveaway', case=False)].head()

,title,author,publication_date,source,url,summary,content,tags,categories,images
id,,,,,,,,,,
672950739902a3465058b1c4,"Biden-Harris EPA funding 'radical, left-leanin...","Aubrie Spady ,Fox News ,Aubrie Spady Is A Writ...",NaT,FOX News,https://www.foxnews.com/politics/biden-harris-...,,The Biden-Harris administration is being put o...,NULL,NULL,https://a57.foxnews.com/cf-images.us-east-1.pr...
67294e199902a3465058b1ba,Judge declines to block Musk’s $1 million vote...,"Lauren Feiner ,A Senior Policy Reporter At The...",2024-11-04,The Verge,https://www.theverge.com/2024/11/4/24288183/mu...,,Elon Musk’s America PAC can move forward with ...,NULL,NULL,https://cdn.vox-cdn.com/thumbor/rodkS6uNARTTLb...
672946ef9902a3465058b1a4,Judge denies Philadelphia DA's request to bloc...,"Abc News ,More ,Election Updates ,Trump To Hos...",NaT,ABC News,https://abcnews.go.com/Politics/elon-musk-pac-...,,A Philadelphia judge is allowing Elon Musk’s A...,NULL,NULL,https://i.abcnewsfe.com/a/a150a26a-eef0-4ebb-a...
672946889902a3465058b1a2,Judge denies Philadelphia DA's request to bloc...,Abc News,NaT,ABC News,https://abcnews.go.com/Politics/elon-musk-pac-...,,A Philadelphia judge is allowing Elon Musk’s A...,NULL,NULL,https://i.abcnewsfe.com/a/a150a26a-eef0-4ebb-a...
6729446c9902a3465058b198,Judge denies Philadelphia DA's request to bloc...,Abc News,NaT,ABC News,https://abcnews.go.com/Politics/elon-musk-pac-...,,A Philadelphia judge is allowing Elon Musk’s A...,NULL,NULL,https://i.abcnewsfe.com/a/a150a26a-eef0-4ebb-a...


In [14]:
# Load 5 rows with the word 'election' in the content from df
df[df['content'].str.contains('election', case=False)].tail()

,title,author,publication_date,source,url,summary,content,tags,categories,images
id,,,,,,,,,,
6724ba5a9aa896701328e02c,Jennifer Lopez Endorses Kamala Harris for Pres...,NULL,2024-11-01,TMZ,https://www.tmz.com/2024/11/01/jennifer-lopez-...,,Play video content\n\nKamala Harris trotted ou...,NULL,NULL,https://imagez.tmz.com/image/92/16by9/2024/11/...
6724ba5a9aa896701328e02a,"Trump, using violent language, attacks 'war ha...","Npr Washington Desk ,November ,Am Et",2024-11-01,NPR,https://www.npr.org/2024/11/01/g-s1-31443/trum...,,"Trump, using violent language, attacks 'war ha...","trump ,Liz Cheney",NULL,https://media.npr.org/chrome/programs/logos/al...
6724ba599aa896701328e029,Brave’s built-in VPN is updated with new featu...,NULL,NaT,Brave Blog,https://brave.com/blog/vpn-relaunch/,,"October 30, 2024 — Today, Brave is excited to ...",NULL,NULL,https://brave.com/static-assets/images/optimiz...
6724ba559aa896701328e023,5 mysteries that might determine the 2024 elec...,Christian Paz,NaT,Vox,https://www.vox.com/2024-elections/381399/elec...,,"is a senior politics reporter at Vox, where he...",NULL,NULL,https://platform.vox.com/wp-content/uploads/si...
6724ba559aa896701328e022,Working-Class Voters Are Pivotal. Both Candida...,Jeanna Smialek,2024-11-01,The New York Times,https://www.nytimes.com/2024/11/01/business/ec...,,Bernadette Daywalt had yet to decide whom to v...,NULL,NULL,https://static01.nyt.com/images/2024/10/30/mul...


In [15]:
# Load 5 rows with the word 'Rug Pull' in the content from df
df[df['content'].str.contains('Rug Pull', case=False)].tail()

,title,author,publication_date,source,url,summary,content,tags,categories,images
id,,,,,,,,,,
672921ad9902a3465058b102,"Binance, Coinbase and Tron Founders Trade Shot...","Decrypt ,Ryan S. Gladwin",2024-11-04 19:32:09,Decrypt,https://decrypt.co/290048/binance-coinbase-tro...,,"Decrypt’s Art, Fashion, and Entertainment Hub....",NULL,NULL,https://img.decrypt.co/insecure/rs:fit:3840:0:...
672921819902a3465058b101,"Binance, Coinbase, and Tron Founders Trade Sho...","Decrypt ,Ryan S. Gladwin",2024-11-04 19:32:09,Decrypt,https://decrypt.co/290048/binance-coinbase-tro...,,"Decrypt’s Art, Fashion, and Entertainment Hub....",NULL,NULL,https://img.decrypt.co/insecure/rs:fit:3840:0:...
6727be3f9902a3465058ad35,"Mutant Ape NFT Creator To Forgo $1.4 Million, ...","Semilore Faleti ,Semilore Faleti Works As A Cr...",NaT,Bitcoinist,https://bitcoinist.com/mutant-ape-nft-creator-...,,"In an interesting development, the creator of ...",NULL,NULL,https://bitcoinist.com/wp-content/uploads/2024...
6724c9c69aa896701328e087,"NFT Developers Plead Guilty to $400,000 'Rug P...","Decrypt ,Ryan S. Gladwin",2024-11-01 12:28:54,Decrypt,https://decrypt.co/289513/nft-developers-guilt...,,"Decrypt’s Art, Fashion, and Entertainment Hub....",NULL,NULL,https://cdn.decrypt.co/resize/1024/height/512/...


In [16]:
df[df.index == '67294e199902a3465058b1ba']['title'].values[0]

'Judge declines to block Musk’s $1 million voter giveaways'

In [17]:
df[df.index == '6724ba5a9aa896701328e02c']['title'].values[0]

'Jennifer Lopez Endorses Kamala Harris for President, Blasts Trump'

In [18]:
df[df.index == '6724c9c69aa896701328e087']['title'].values[0]

"NFT Developers Plead Guilty to $400,000 'Rug Pull’, Laundering and Wire Fraud"

##### Selected news events
- 67294e199902a3465058b1ba - Judge declines to block Musk’s $1 million voter giveaways
- 6724ba5a9aa896701328e02c - Jennifer Lopez Endorses Kamala Harris for President, Blasts Trump
- 6724c9c69aa896701328e087 - NFT Developers Plead Guilty to $400,000 'Rug Pull’, Laundering and Wire Fraud

In [8]:
case1_id = '67294e199902a3465058b1ba'
case2_id = '6724ba5a9aa896701328e02c'
case3_id = '6724c9c69aa896701328e087'

## A simple approach (Trial 1)

- Just getting embeddings of the content of each article and using a vector database to search the similars.

- Embedding mode: `all-mpnet-base-v2`
- Vector database: `ChromaDB`

In [9]:
from scripts.db_1 import Database
from scripts.embeddings_trial1 import Embeddings

db = Database('articles')
embed_model = Embeddings()

In [21]:
for row in tqdm(df.iterrows(), desc="Adding articles to Database", total=df.shape[0]):
  article = row[1]
  embedding = embed_model.generate_embeddings(article_content=article['content'])
  if embedding is None:
    raise Exception("Embedding is None")
  db.insert_document_by_embedding([row[0]], [embedding], [row[0]])

Adding articles to Database: 100%|██████████| 2080/2080 [23:52<00:00,  1.45it/s]


In [22]:
embedding.shape

(768,)

- Case 1

In [10]:
# Case 1
case1 = db.get_document([case1_id])

search_results = db.search_nearest_documents(case1['embeddings'], 30)


In [11]:
case1

{'ids': ['67294e199902a3465058b1ba'],
 'embeddings': array([[ 2.64897533e-02,  1.25086278e-01, -2.42463537e-02,
          8.36910158e-02, -1.15052313e-02, -7.18016468e-04,
         -6.85784668e-02,  6.14656415e-03, -2.08123270e-02,
         -4.92500290e-02,  3.46640646e-02, -2.28095539e-02,
          1.57090388e-02, -1.13801667e-02, -1.34485296e-03,
         -1.19611677e-02, -9.04761720e-03, -5.18046096e-02,
          5.97322620e-02, -6.35935813e-02, -4.12254594e-02,
         -3.02789751e-02, -7.14915060e-03,  1.41456593e-02,
          4.54695225e-02,  2.22552381e-02,  1.79047789e-02,
         -1.28367255e-02, -3.22326459e-02, -1.61981564e-02,
          1.23754377e-02,  2.16254368e-02, -4.42019068e-02,
          1.24177756e-03,  2.11034717e-06, -5.15646338e-02,
          2.82377340e-02,  6.97486941e-03,  3.14783938e-02,
          1.44725954e-02, -1.20117068e-02,  6.50139302e-02,
         -4.64815088e-02,  1.03424378e-02, -3.69942212e-03,
         -5.05360663e-02,  6.81113126e-03,  3.86

In [12]:
search_results

{'ids': [['67294e199902a3465058b1ba',
   '672917219902a3465058b0ba',
   '6729446c9902a3465058b198',
   '6729083f9902a3465058b069',
   '672912229902a3465058b098',
   '6729136e9902a3465058b0a1',
   '672912619902a3465058b09b',
   '672946ef9902a3465058b1a4',
   '67292daa9902a3465058b12e',
   '67293a219902a3465058b16b',
   '672936bb9902a3465058b155',
   '67292ecd9902a3465058b133',
   '67294bcb9902a3465058b1b2',
   '67293b969902a3465058b170',
   '67293e959902a3465058b17e',
   '6728f3ff9902a3465058b00b',
   '672953fd9902a3465058b1d0',
   '67291ab79902a3465058b0d5',
   '67291b739902a3465058b0df',
   '672918349902a3465058b0c3',
   '6724d5fd9aa896701328e0c5',
   '6724d8139aa896701328e0d2',
   '6728a2b89902a3465058aee2',
   '672902cd9902a3465058b04f',
   '67279a629902a3465058acde',
   '6728b0e49902a3465058af0b',
   '672722cf9902a3465058abbe',
   '6728d7e59902a3465058af81',
   '67272d4d9902a3465058abf0',
   '672508929aa896701328e1e7']],
 'embeddings': [array([[ 0.02648975,  0.12508628, -0.02424635

In [13]:
search_results['embeddings'][0][0]

array([ 2.64897533e-02,  1.25086278e-01, -2.42463537e-02,  8.36910158e-02,
       -1.15052313e-02, -7.18016468e-04, -6.85784668e-02,  6.14656415e-03,
       -2.08123270e-02, -4.92500290e-02,  3.46640646e-02, -2.28095539e-02,
        1.57090388e-02, -1.13801667e-02, -1.34485296e-03, -1.19611677e-02,
       -9.04761720e-03, -5.18046096e-02,  5.97322620e-02, -6.35935813e-02,
       -4.12254594e-02, -3.02789751e-02, -7.14915060e-03,  1.41456593e-02,
        4.54695225e-02,  2.22552381e-02,  1.79047789e-02, -1.28367255e-02,
       -3.22326459e-02, -1.61981564e-02,  1.23754377e-02,  2.16254368e-02,
       -4.42019068e-02,  1.24177756e-03,  2.11034717e-06, -5.15646338e-02,
        2.82377340e-02,  6.97486941e-03,  3.14783938e-02,  1.44725954e-02,
       -1.20117068e-02,  6.50139302e-02, -4.64815088e-02,  1.03424378e-02,
       -3.69942212e-03, -5.05360663e-02,  6.81113126e-03,  3.86694111e-02,
       -2.73975600e-02, -4.49131727e-02,  1.90080609e-02,  5.22376113e-02,
        3.27154100e-02, -

In [16]:
def articles_similarity_summary(case_id, n, flag_value=0.5):
  case = db.get_document([case_id])

  search_results = db.search_nearest_documents(case['embeddings'], n)

  print(f"\033[1m{'ID':<30}| {'Title':<90}| {'Source':<30}| {'Published Date':<30}| {'Similarity':<30}\033[0m")
  print(f"{'-'*30}+-{'-'*90}+-{'-'*30}+-{'-'*30}+-{'-'*30}")
  for i, result_id in enumerate(search_results['ids'][0]):
    embedding = search_results['embeddings'][0][i]
    doc_title = df[df.index == result_id]['title'].values[0]
    doc_title = doc_title[:70] + '...' if len(doc_title) > 70 else doc_title
    doc_source = df[df.index == result_id]['source'].values[0]
    doc_date = df[df.index == result_id]['publication_date'].values[0]
    similarity = embed_model.model.similarity(case['embeddings'], embedding)
    if similarity.item() > flag_value:
      print(f'\033[92m{result_id:<30}| {doc_title:<90}| {doc_source:<30}| {doc_date:<30}| {similarity.item():<30.6f}\033[0m')
    else:
      print(f'{result_id:<30}| {doc_title:<90}| {doc_source:<30}| {doc_date:<30}| {similarity.item():<30.6f}')

In [19]:
articles_similarity_summary(case1_id, 30)

ID                            | Title                                                                                     | Source                        | Published Date                | Similarity                    
------------------------------+-------------------------------------------------------------------------------------------+-------------------------------+-------------------------------+-------------------------------
67294e199902a3465058b1ba      | Judge declines to block Musk’s $1 million voter giveaways                                 | The Verge                     | 2024-11-04T00:00:00.000000000 | 1.000000                      
672917219902a3465058b0ba      | Elon Musk’s PAC admits $1 million voter giveaways aren’t ‘random’                         | The Verge                     | 2024-11-04T00:00:00.000000000 | 0.925632                      
6729446c9902a3465058b198      | Judge denies Philadelphia DA's request to block Elon Musk's $1 million...                 | 

In [20]:
articles_similarity_summary(case2_id, 120)

ID                            | Title                                                                                     | Source                        | Published Date                | Similarity                    
------------------------------+-------------------------------------------------------------------------------------------+-------------------------------+-------------------------------+-------------------------------
6724ba5a9aa896701328e02c      | Jennifer Lopez Endorses Kamala Harris for President, Blasts Trump                         | TMZ                           | 2024-11-01T00:00:00.000000000 | 1.000000                      
6724d0049aa896701328e0a2      | Jennifer Lopez Endorses Kamala Harris for President, Blasts Trump                         | TMZ                           | 2024-11-01T00:00:00.000000000 | 0.996461                      
6724faf89aa896701328e18d      | Jennifer Lopez goes after Trump comic's Puerto Rican dig as she asks v...                 | 

In [21]:
articles_similarity_summary(case3_id, 50)

ID                            | Title                                                                                     | Source                        | Published Date                | Similarity                    
------------------------------+-------------------------------------------------------------------------------------------+-------------------------------+-------------------------------+-------------------------------
6724c9c69aa896701328e087      | NFT Developers Plead Guilty to $400,000 'Rug Pull’, Laundering and Wir...                 | Decrypt                       | 2024-11-01T12:28:54.000000000 | 1.000000                      
6724f7d29aa896701328e180      | Ethereum Gaming Token IMX Plunges After Immutable Reveals SEC Threat                      | Decrypt                       | 2024-11-01T15:46:08.000000000 | 0.722982                      
6724f8129aa896701328e182      | Ethereum Gaming Token IMX Plunges After Immutable Reveals SEC Threat                      | 

In [22]:
df.columns

Index(['title', 'author', 'publication_date', 'source', 'url', 'summary',
       'content', 'tags', 'categories', 'images'],
      dtype='object')

In [25]:
# Clear collection
# doc_ids = df.index.to_list()
# db.delete_document(doc_ids)